In [2]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

docs_service = build("docs", "v1")



In [3]:
def create_document(title: str) -> str:
    try:
        doc = docs_service.documents().create(
            body={"title": title}
        ).execute()
        return doc["documentId"]
    except HttpError as e:
        raise RuntimeError(f"Failed to create document: {e}")


In [4]:
MARKDOWN_TEXT = """# Product Team Sync - May 15, 2023
## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets
"""


In [5]:
def insert_text(requests, text):
    requests.append({
        "insertText": {
            "location": {"index": 1},
            "text": text
        }
    })


def apply_paragraph_style(requests, start, end, style):
    requests.append({
        "updateParagraphStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "paragraphStyle": {"namedStyleType": style},
            "fields": "namedStyleType"
        }
    })


In [9]:
import re

def markdown_to_requests(markdown: str):
    requests = []
    index = 1

    for line in markdown.splitlines():
        if line.startswith("# "):
            text = line[2:] + "\n"
            insert_text(requests, text)
            apply_paragraph_style(
                requests, index, index + len(text), "HEADING_1"
            )
            index += len(text)

        elif line.startswith("## "):
            text = line[3:] + "\n"
            insert_text(requests, text)
            apply_paragraph_style(
                requests, index, index + len(text), "HEADING_2"
            )
            index += len(text)

        elif line.startswith("### "):
            text = line[4:] + "\n"
            insert_text(requests, text)
            apply_paragraph_style(
                requests, index, index + len(text), "HEADING_3"
            )
            index += len(text)

        elif re.match(r"- \[ \]", line):
            text = line[6:] + "\n"
            insert_text(requests, text)
            requests.append({
                "createParagraphBullets": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(text)
                    },
                    "bulletPreset": "BULLET_CHECKBOX"
                }
            })
            index += len(text)

        elif line.startswith(("-", "*")):
            text = line[2:] + "\n"
            insert_text(requests, text)
            requests.append({
                "createParagraphBullets": {
                    "range": {
                        "startIndex": index,
                        "endIndex": index + len(text)
                    },
                    "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"
                }
            })
            index += len(text)

        elif line.strip() == "---":
            text = "\n"
            insert_text(requests, text)
            index += len(text)

        else:
            text = line + "\n"
            insert_text(requests, text)
            index += len(text)

    return requests


In [10]:
def style_mentions(requests, doc_text):
    for match in re.finditer(r"@\w+", doc_text):
        requests.append({
            "updateTextStyle": {
                "range": {
                    "startIndex": match.start() + 1,
                    "endIndex": match.end() + 1
                },
                "textStyle": {"bold": True},
                "fields": "bold"
            }
        })


def style_footer(requests, start, end):
    requests.append({
        "updateTextStyle": {
            "range": {"startIndex": start, "endIndex": end},
            "textStyle": {
                "italic": True,
                "foregroundColor": {
                    "color": {"rgbColor": {"red": 0.5, "green": 0.5, "blue": 0.5}}
                }
            },
            "fields": "italic,foregroundColor"
        }
    })


In [11]:
doc_id = create_document("Product Team Sync")

requests = markdown_to_requests(MARKDOWN_TEXT)

docs_service.documents().batchUpdate(
    documentId=doc_id,
    body={"requests": requests}
).execute()

print(f"Document created: https://docs.google.com/document/d/{doc_id}")


Document created: https://docs.google.com/document/d/1Hh4OvVGl0O2H6JC0Sxf_MUV_6nqKjnecKkx3koJr3vk
